In [4]:
class variable:
    def  __init__(self, name, isNegative):
        self.name = name
        self.isNegative =isNegative
    def value(self, name_map):
        if self.name not in name_map.keys():
            return "undefined"
        if self.isNegative is False:
            return name_map[self.name]
        if self.isNegative:
            if  name_map[self.name] == 1:
                return 0
            else:
                return 1
    def  name(self):  #返回变量名称
        return  self.name
    def __str__(self):
        if self.isNegative:
            return self.name + "^"
        return self.name 
class clause:  #带括号的模块
    def  __init__(self, variables):
        self.variables = variables  #一个模块是包含多个变量的集合
    def  reduce(self, name_map) :#在设定给定变量的值后调整模块当前变量
        variables_left = []
        for variable in self.variables:
            if variable.value(name_map) == 1:  #如果变量值设置为1，当前模块则无需考虑
                return  None
            if variable.value(name_map) == "undefined":
                variables_left.append(variable)
        return clause(variables_left)
    def  value(self , name_map):  #获取模块的值
        zero_var_count = 0
        for variable in self.variables:
            if variable.value(name_map) == 1:
                return 1
            if variable.value(name_map) == 0:
                zero_var_count += 1
        if zero_var_count == len(self.variables):
            return 0
        return "undefined"
    def __str__(self):
        if len(self.variables) == 0:
            return "(*)"
        clause_s = "("
        for i in range(len(self.variables)):
            clause_s +=  str(self.variables[i])
            if i != len(self.variables) - 1:
                clause_s += "|"
        clause_s += ")"
        return clause_s
class  expression:  #对应表达式
    def __init__(self, clauses, name_map):
        self.clauses = clauses
        self.name_map = name_map
    def value(self):
        if len(self.clauses) == 0: #当一个模块的值为1时它对应的表达式就是None，因此当表达式中Clause的个数为0时意味着每个clause都取值1
            return 1
        for clause in self.clauses:
            if clause.value(self.name_map) == 0:  #只要有一个模块的值为0整个表达式的值就为0
                return 0
            if clause.value(self.name_map) == "undefined":
                return  "undefined"
            if clause.value(self.name_map) == 1:
                return 1
        return 0
    def reduce_by_name(self, var_name):
        expressions = []
        split_count = 0
        while split_count < 2:  #根据变量两种取值从当前节点分出两个子节点
            name_map_copy = self.name_map.copy()
            name_map_copy[var_name] = split_count
            new_clauses = []
            split_count += 1
            for clause in self.clauses:
                reduce_clause = clause.reduce(name_map_copy)
                if reduce_clause != None:
                    new_clauses.append(reduce_clause)
            expressions.append(expression(new_clauses, name_map_copy))
        return expressions
    def  __str__(self):
        express_s = ""
        for i in range(len(self.clauses)):
            if str(self.clauses[i]) == "":
                continue
            express_s += str(self.clauses[i])
            if i != len(self.clauses) - 1:
                express_s += "&"
        if express_s == "":
            express_s = "(1)"
        return express_s
    def get_name_map(self):
        return  self.name_map
variable_names = ["w", "x", "y", "z"] #构造公式(3)对应的表达式
clause_1 = clause([variable("w", False), variable("x", False), variable("y", True), variable("z", False)])
clause_2 = clause([variable("w", False), variable("x", False)])
clause_3 = clause([variable("x", True), variable("y", False)])
clause_4 = clause([variable("y", False), variable("z", True)])
clause_5 = clause([variable("z", False), variable("w", True)])
clause_6 = clause([variable("w", True), variable("z", True)])
S = []
variable_name_pointer = 0
S.append((expression([clause_1,clause_2,clause_3,clause_4,clause_5,clause_6], {}), variable_name_pointer))
while len(S) > 0:
    e, variable_name_pointer = S.pop()  #从集合中取出表达式实现进一步探索
    print("current expression is: ", e)
    if e.value() == 1:  #如果表达式取值1打印出当前变量的取值情况
        print("find variable assigment:")
        e_map = e.get_name_map()
        for v in e_map.keys():
            print("{0} : {1}".format(v, e_map[v]))
        break
    elif  e.value() == "undefined" and variable_name_pointer < len(variable_names):
        variable_name = variable_names[variable_name_pointer]  #获取要赋值的变量名
        variable_name_pointer += 1
        print("split current expression by vairable: ", variable_name)
        es = e.reduce_by_name(variable_name) #将变量分别赋值为0和1然后将当前表达式分解成形式更简单的表达式
        print("splited expression are: ")
        for ee in es:
            map = ee.get_name_map()
            print("{0} with variable {1} set to {2} and expression value is {3}: ".format(ee, variable_name, map[variable_name], ee.value()))
            if ee.value() != 0:  #如果当前表达式取值0就不用加入集合
                S.append((ee, variable_name_pointer))

current expression is:  (w|x|y^|z)&(w|x)&(x^|y)&(y|z^)&(z|w^)&(w^|z^)
split current expression by vairable:  w
splited expression are: 
(x|y^|z)&(x)&(x^|y)&(y|z^) with variable w set to 0 and expression value is undefined: 
(x^|y)&(y|z^)&(z)&(z^) with variable w set to 1 and expression value is undefined: 
current expression is:  (x^|y)&(y|z^)&(z)&(z^)
split current expression by vairable:  x
splited expression are: 
(y|z^)&(z)&(z^) with variable x set to 0 and expression value is undefined: 
(y)&(y|z^)&(z)&(z^) with variable x set to 1 and expression value is undefined: 
current expression is:  (y)&(y|z^)&(z)&(z^)
split current expression by vairable:  y
splited expression are: 
(*)&(z^)&(z)&(z^) with variable y set to 0 and expression value is 0: 
(z)&(z^) with variable y set to 1 and expression value is undefined: 
current expression is:  (z)&(z^)
split current expression by vairable:  z
splited expression are: 
(*) with variable z set to 0 and expression value is 0: 
(*) with varia

In [6]:
variables = ['s', 'j', 'b', 'p', 'S', 'J', 'B', 'H']  #公式中所有变量
clauses = [(['s'], 'B'),  #对应推导模块
            (['j'], 'J'),
            (['s', 'J'], 'p'),
            (['b', 'j'], 'S'),
            (['B', 'J'], 'S'),
            ([], 's'),
            ([], 'j')]
negatives = [['b', 'H']]

In [9]:
def greedy_hornsat(variables, clauses, negates):
    assignment = dict((v, False) for v in variables)  #将所有变量初始化为0
    working = set(t[1] for t in clauses if not len(t[0]))  #获得所有推导模块箭头右边变量
    while working:  #依次将赋值不满足的推导模块右边变量设置为1
        v = working.pop()
        assignment[v] = True
        vclauses = [ c for c in clauses if v in c[0]]
        for vclause in vclauses:
            '''
            如果推导模块箭头左边变量取值1那么该变量可以删除掉，
            如果左边所有变量都被删除这意味着推导模块左边所有变量都取值1因此需要右边变量也取值1
            '''
            vclause[0].remove(v)  
            if not vclause[0]:#推导模块左边变量都取值1，将右边变量加入集合以便下次循环将它设置为1
                working.add(vclause[1])
    for negate in negates:  #检验取反模块是否都能满足
        has_one_false = False
        for v in negate:  #取反模块要满足要求至少一个变量取0
            if assignment[v] == False:
                has_one_false = True
                break
        if has_one_false != True:
            print('No valid assigment exist!')
            return None
    return assignment

In [11]:
assignment = greedy_hornsat(variables, clauses,negatives)
print(assignment)

{'s': True, 'j': True, 'b': False, 'p': True, 'S': True, 'J': True, 'B': True, 'H': False}


In [12]:
import sys 
class ElementSet:
    def  __init__(self, i):
        self.setIndex = i 
        self.elementCount= 0
        self.elementDict = {}
    def addElement(self, e):
        if self.elementDict.get(e, None) is None:
            self.elementCount += 1
            self.elementDict[e] = 1
            e.addSet(self)
    def delElement(self, e):
      element = self.elementDict.get(e, None)
      if element is not None and element == 1:
          self.elementCount -= 1
          self.elementDict[e] = 0
                
    def getElementCount(self):
        return self.elementCount
    def printElementSet(self):
        print("[ ", end="")
        for e in self.elementDict.keys():
            if self.elementDict[e] == 1:
                print("{0} ".format(e.val), end="")
        print(" ]")
    def setCovered(self):
        #当集合被选中红，要把所包含元素覆盖掉
        for e in self.elementDict.keys():
            if self.elementDict[e] == 1:
                e.covered()
class Element:
    def __init__(self, val):
        self.val = val 
        self.sets = []
    def addSet(self, set):
        #把元素加入集合
        self.sets.append(set)
    def covered(self):
        #当元素被覆盖后，包含它的集合的元素个数要减1
        for set in self.sets:
            set.delElement(self)

In [13]:
eCount = 30
sCount = 6
eSet = []
sSet = []
elementNum = eCount
import random
#把eCount个元素随机分配到sCount个集合中
for i in range(sCount):
    s = ElementSet(i)
    sSet.append(s)
for i in range(1, eCount+1):
    e = Element(i)
    eSet.append(e)
    #随机的把元素分配到几个集合
    setGot = random.randint(1, sCount)
    while setGot > 0:
        setSelected = {}
        sel = random.randint(0, sCount-1)
        if setSelected.get(sel, None) is None:
            #记录下已经被选中的集合号
            setSelected[sel] = 1
            sSet[sel].addElement(e)
            setGot -= 1
def printAllSets():
    for s in sSet:
        s.printElementSet()
printAllSets()
while elementNum > 0:
    sel = 0;
    eCount = 0
    for i in range(len(sSet)):
        #选取当前含有未覆盖元素最大的集合
        if sSet[i].getElementCount() > eCount:
            eCount = sSet[i].getElementCount()
            sel = i 
    print("select set {0}, with element count {1}".format(sel, sSet[sel].getElementCount()))
    elementNum -= sSet[sel].getElementCount()        
    sSet[sel].setCovered()

[ 3 4 5 6 9 11 13 14 15 25 27 28 29 30  ]
[ 1 6 7 9 10 11 18 22 27 28 29 30  ]
[ 4 5 8 9 11 19 21 22 23 26 27 29  ]
[ 2 3 5 7 8 9 10 18 19 30  ]
[ 1 4 10 11 12 16 17 18 19 22 26 27 28 29  ]
[ 4 10 19 20 22 23 24 26 27 29 30  ]
select set 0, with element count 14
select set 4, with element count 9
select set 2, with element count 3
select set 3, with element count 2
select set 5, with element count 2


In [29]:
class Vertex:
    def __init__(self, name):
        self.parent = self  #parent对应节点指针,默认指向它自己
        self.rank = 0 #rank表示子节点的个数
        self.name = name #节点名字
    def  point_to(self, v):  #将指针指向另一个节点
        if v is not None:
            self.parent = v
    def  get_parent(self):  #获取节点指针指向的节点
        return self.parent
    def find(self): #获取集合中的根节点
        x = self
        while x.get_parent() != x:
            x = x.get_parent()
        return x
    def set_rank(self, r):
        self.rank = r
    def get_rank(self):
        return self.rank
    def get_name(self):
        return self.name
def  union(x, y): #x,y是两个连接点集合中的点，该函数将它们所在的集合合并成一个集合
    root_x = x.find()
    root_y = y.find()
    if root_x == root_y:
        return 
    if root_x.get_rank() > root_y.get_rank(): #把高度低的根节点指向高度高的根节点
        root_y.point_to(root_x)
    else:
        root_x.point_to(root_y)
    if root_x.get_rank() == root_y.get_rank(): 
        rank = root_y.get_rank()
        root_y.set_rank(rank + 1)
class  Edge:
    def __init__(self, v1, v2, val):#v1,v2对应边的连接点,val对应边的数值
        self.v1 = v1
        self.v2 = v2
        self.edge_value = val
    def  get_points(self):
        return self.v1, self.v2
    def  get_edge_value(self):
        return self.edge_value
    def  print_edge(self): #打印边的信息
        print("{0}---{1} with value {2}".format(self.v1.get_name(), self.v2.get_name(), self.edge_value))
    def  __str__(self):
        return "{0}---{1}".format(self.v1.get_name(), self.v2.get_name())
vertexes = []  #所有点的集合
for i in range(0, 8):  #依据图4-7构建节点
    name = chr(ord("A") + i)
    vertexes.append(Vertex(name))
def  get_vertex_by_name(name):  #根据给定名字从点集合中返回节点
    for v in vertexes:
        if v.get_name() == name:
            return v
    return None
def  reset_vertexs(vertexes):
    for v in vertexes:
        v.parent = v
def  mini_spanning_tree(E):
    X = []
    cost = 0
    for edge in E: #每次选出一条数值最小又能连接切割的边
        u,v = edge.get_points()
        if u.find() != v.find(): #如果边的两个点不在同一个连接点集合内那么就选择该边
            X.append(edge)
            edge.print_edge()
            union(u, v) #将该线段的两点对应的点集合合并
            cost += edge.get_edge_value()
    return cost

In [30]:
E = []
adjacent_map = {} #记录给定点的相邻点
adjacent_map["A"] = ["A","F", "B", "H"]
adjacent_map["B"] = ["A", "C", "E"]
adjacent_map["C"] = ["B", "D", "H"]
adjacent_map["D"] = ["C", "E", "G"]
adjacent_map["E"] = ["D", "B", "F"]
adjacent_map["F"] = ["E", "A", "G"]
adjacent_map["G"] = ["F", "D", "H"]
adjacent_map["H"] = ["G", "C", "A"]
vertex_edge_map = {}  #给定两个节点返回他们对应的边
E_AA = Edge(get_vertex_by_name("A"), get_vertex_by_name("A"), 0)
E.append(E_AA)
vertex_edge_map[("A", "A")] = E_AA
E_AB = Edge(get_vertex_by_name("A"), get_vertex_by_name("B"), 2)
E.append(E_AB)
vertex_edge_map[("A", "B")] = E_AB
vertex_edge_map[("B", "A")] = E_AB
E_AH = Edge(get_vertex_by_name("A"), get_vertex_by_name("H"), 1)
E.append(E_AH)
vertex_edge_map[("A", "H")] = E_AH
vertex_edge_map[("H", "A")] = E_AH
E_AF = Edge(get_vertex_by_name("A"), get_vertex_by_name("F"), 1)
E.append(E_AF)
vertex_edge_map[("A", "F")] = E_AF
vertex_edge_map[("F", "A")] = E_AF
E_BE = Edge(get_vertex_by_name("B"), get_vertex_by_name("E"), 1)
E.append(E_BE)
vertex_edge_map[("B", "E")] = E_BE
vertex_edge_map[("E", "B")] = E_BE
E_BC = Edge(get_vertex_by_name("B"), get_vertex_by_name("C"), 1)
E.append(E_BC)
vertex_edge_map[("B", "C")] = E_BC
vertex_edge_map[("C", "B")] = E_BC
E_CD = Edge(get_vertex_by_name("C"), get_vertex_by_name("D"), 1)
E.append(E_CD)
vertex_edge_map[("C", "D")] = E_CD
vertex_edge_map[("D", "C")] = E_CD
E_CH = Edge(get_vertex_by_name("C"), get_vertex_by_name("H"), 5)
E.append(E_CH)
vertex_edge_map[("C", "H")] = E_CH
vertex_edge_map[("H", "C")] = E_CH
E_DE = Edge(get_vertex_by_name("D"), get_vertex_by_name("E"), 2)
E.append(E_DE)
vertex_edge_map[("D", "E")] = E_DE
vertex_edge_map[("E", "D")] = E_DE
E_DG = Edge(get_vertex_by_name("D"), get_vertex_by_name("G"), 1)
E.append(E_DG)
vertex_edge_map[("D", "G")] = E_DG
vertex_edge_map[("G", "D")] = E_DG
E_EF = Edge(get_vertex_by_name("E"), get_vertex_by_name("F"), 1)
E.append(E_EF)
vertex_edge_map[("E", "F")] = E_EF
vertex_edge_map[("F", "E")] = E_EF
E_FG = Edge(get_vertex_by_name("F"), get_vertex_by_name("G"), 1)
E.append(E_FG)
vertex_edge_map[("F", "G")] = E_FG
vertex_edge_map[("G", "F")] = E_FG
E_GH = Edge(get_vertex_by_name("G"), get_vertex_by_name("H"), 1)
E.append(E_GH)
vertex_edge_map[("G", "H")] = E_GH
vertex_edge_map[("H", "G")] = E_GH
E.sort(key = lambda x : x.get_edge_value())  #对边按照数值大小进行排序

In [31]:
import sys
def  print_path(paths):
    paths_s = ""
    for p in paths:
        paths_s += str(p) + ","
    print(paths_s)
def path_str(paths):
    paths_s = ""
    for p in paths:
        paths_s += str(p) + ","
    return paths_s
def  find_shortest_circle(E, start, vertex_edge_map, adjacent_map, vertexes):
    start_path = vertex_edge_map[(start, start)]
    S = [([start_path], 0)] #([组成当前路径节的所有边],路径长度)
    best_so_far = sys.maxsize
    shortest_circle_length = sys.maxsize
    shortest_circle = None
    while  len(S) > 0:
        paths, paths_len = S.pop()
        print("current path is :")
        print_path(paths)
        last_edge = paths[-1]
        v_prev, v_last = last_edge.get_points()  #获取最后一条边的两个节点
        if len(paths) > 1: #由于边没有方向因此我们需要确定v_prev,v_last哪个节点是边的最后节点
            prev_edge = paths[-2]  #倒数第二条边
            if v_last in prev_edge.get_points():
                temp = v_last
                v_last = v_prev
                v_prev = temp
        vertexs_with_v =  adjacent_map[v_last.get_name()]  #获得与最后节点相邻的点
        vertexes_in_current_path = []
        for e in paths:  #记录当前路径包含的节点
            v1, v2 = e.get_points()
            if v1.get_name() not in  vertexes_in_current_path:
                vertexes_in_current_path.append(v1.get_name())
            if v2.get_name() not in vertexes_in_current_path:
                vertexes_in_current_path.append(v2.get_name())
        for vertex in vertexs_with_v:
            if vertex in vertexes_in_current_path:  #下一个节点必须是新节点，不能使已经遍历过的节点
                continue
            print("vertex to be checked:", vertex)
            last_vertex_name = v_last.get_name()
            new_edge = vertex_edge_map[(last_vertex_name, vertex)]
            paths_cp = paths.copy()
            paths_cp.append(new_edge) #将与最后一个节点相连的边加入
            paths_len_cp = paths_len + new_edge.get_edge_value()
            edges_from_last_to_V_S = []
            vertex_with_last = adjacent_map[vertex]
            for v in vertex_with_last:  #记录最后一个节点与集合V-S连接的所有边
                e = vertex_edge_map[(vertex, v)]
                if e not in paths_cp:
                    edges_from_last_to_V_S.append(e)
            edges_from_start_to_V_S = []
            vertex_with_first = adjacent_map[start]
            for v in vertex_with_first:  #记录起始点到集合V-S的所有边
                e = vertex_edge_map[(start, v)]
                if e not in paths_cp:
                    edges_from_start_to_V_S.append(e)
            edges_no_in_V_S = []
            for edge in paths_cp:  #所有在集合S中的点发出的边都不属于集合V-S中点构成的图形
                v1, v2 = edge.get_points()
                points_with_v1 = adjacent_map[v1.get_name()]
                for point_name in points_with_v1:
                    e = vertex_edge_map[(v1.get_name(), point_name)]
                    if e not in edges_no_in_V_S:
                        edges_no_in_V_S.append(e)
                points_with_v2 = adjacent_map[v2.get_name()]
                for point_name in points_with_v2:
                    e = vertex_edge_map[(v2.get_name(), point_name)]
                    if e not in edges_no_in_V_S:
                        edges_no_in_V_S.append(e)
            V_S = [p for p in E if p not in edges_no_in_V_S]  #获得由点集合V-S形成图形的边
            print("paths not in set V-S:")
            print_path(edges_no_in_V_S)
            print("paths in set V-S: ")
            print_path(V_S)
            print("edges in mini spanning tree of set V-S are:")
            v_s_spanning_tree = mini_spanning_tree(V_S)  #计算V-S节点所形成图的最小生成树长度
            print("mini spanning tree length in V-S is ", v_s_spanning_tree)
            reset_vertexs(vertexes)  #重置节点对象以便下一次计算生成树长度
            shortest_from_start_to_V_S = sys.maxsize
            edge_start_to_V_S = None
            for e in edges_from_start_to_V_S:  #获得起始节点到集合V-S最短距离
                if e.get_edge_value() < shortest_from_start_to_V_S:
                    shortest_from_start_to_V_S = e.get_edge_value()
                    edge_start_to_V_S = e
            shortest_from_last_to_V_S = sys.maxsize
            edge_last_to_V_S = None
            for e in edges_from_last_to_V_S:  #计算末尾节点到集合V-S的最短距离
                if e.get_edge_value() < shortest_from_last_to_V_S:
                    shortest_from_last_to_V_S = e.get_edge_value()
                    edge_last_to_V_S = e
            #计算剩余长度的下限
            circle_lower_bound = shortest_from_last_to_V_S + shortest_from_start_to_V_S  + v_s_spanning_tree + paths_len_cp
            print("current path is {0} , circle contains this path has lower bound of {1}".format(path_str(paths_cp), circle_lower_bound))
            if circle_lower_bound <= best_so_far:  #如果增加该点后回路长度下限不变大，那么将它加入S
                best_so_far = circle_lower_bound
                if best_so_far < shortest_circle_length and (paths_cp, paths_len_cp) not in S:
                    S.append((paths_cp, paths_len_cp))
            if len(V_S) == 0:  #所有节点都在S中,我们找到一条环
                if (vertex, start) in vertex_edge_map.keys():
                    if paths_len_cp + edge_start_to_V_S.get_edge_value() +  edge_last_to_V_S.get_edge_value() <= best_so_far:
                        shortest_circle_length = paths_len_cp  + edge_start_to_V_S.get_edge_value() +  edge_last_to_V_S.get_edge_value()  #找到长度更小的环
                        shortest_circle = paths_cp
                        shortest_circle.append(edge_start_to_V_S)
                        shortest_circle.append(edge_last_to_V_S)
                        print("find a circle with edges: ")
                        print_path(shortest_circle)
                        print("circle length is : ", shortest_circle_length)
    if shortest_circle is not None:
        print("find shortest circle:")
        print_path(shortest_circle)
        print("its length is ", shortest_circle_length)
find_shortest_circle(E, "A", vertex_edge_map, adjacent_map, vertexes)

current path is :
A---A,
vertex to be checked: F
paths not in set V-S:
A---A,A---F,A---B,A---H,E---F,F---G,
paths in set V-S: 
B---E,B---C,C---D,D---G,G---H,D---E,C---H,
edges in mini spanning tree of set V-S are:
B---E with value 1
B---C with value 1
C---D with value 1
D---G with value 1
G---H with value 1
mini spanning tree length in V-S is  5
current path is A---A,A---F, , circle contains this path has lower bound of 8
vertex to be checked: B
paths not in set V-S:
A---A,A---F,A---B,A---H,B---C,B---E,
paths in set V-S: 
C---D,D---G,E---F,F---G,G---H,D---E,C---H,
edges in mini spanning tree of set V-S are:
C---D with value 1
D---G with value 1
E---F with value 1
F---G with value 1
G---H with value 1
mini spanning tree length in V-S is  5
current path is A---A,A---B, , circle contains this path has lower bound of 9
vertex to be checked: H
paths not in set V-S:
A---A,A---F,A---B,A---H,G---H,C---H,
paths in set V-S: 
B---E,B---C,C---D,D---G,E---F,F---G,D---E,
edges in mini spanning tree 

In [8]:
import random
N = 30  #集合S的元素个数
S = []
for i in range(N):
    S.append(random.randint(1, 100))
T = 3  #随机从S中选择给定个数的元素加总得t
indexs = random.sample(range(0, N), T)
print("selected element index: ", indexs)
t = 0
for i in indexs:
    t += S[i]
print("t: ", t)
def  find_subset_sum(S, t):
    L_prev = [0]  #相当于L(i-1)
    for i in range(1, len(S)):
        L = []
        for l in L_prev:  #L(i-1)+x(i)
            L.append(l + S[i])
        L_cur = []  #相当于L(i)
        p = 0
        q = 0
        while p < len(L_prev) or q < len(L): #将L(i-1) 与 L(i-1) + x(i)两个数组的元素合并，同时保证合并后元素不重复且排好序
            if p < len(L_prev) and q < len(L):
                k = min(L_prev[p], L[q])
                if k <= t and k not in L_cur:
                    L_cur.append(k)
                if k == L_prev[p]:
                    p += 1
                else:
                    q += 1
            else:  #将剩余数组中的元素加入L_cur
                LL = []
                if p >= len(L_prev):
                    LL = L
                else:
                    LL = L_prev 
                for l in LL:
                    if l not in L_cur and l <= t:
                        L_cur.append(l)
                break
        L_prev = L_cur 
    return L_prev[len(L_prev) - 1] == t  #L_prev是一个已经排好序的数组，所有元素都不大于t，因此如果有元素等于t，只能是最后一个
exist = find_subset_sum(S, t)
if exist:
    print("there is subset in S sum up to t")
else:
    print("there is no subset in S sump up to t")

selected element index:  [7, 10, 25]
t:  220
there is subset in S sum up to t


In [2]:
def  trim(L_Prev, factor):
    y = L_Prev[0]
    last = y
    L = [y] 
    for i in range(1, len(L_Prev)):
        if L_Prev[i] > last * (1 + factor):
            L.append(L_Prev[i])
            last = L_Prev[i]
    return L 
L_Prev = [10, 11, 12, 15, 20, 21, 22, 23, 24, 29]
L = trim(L_Prev, 0.1)
print(L) 

[10, 12, 15, 20, 23, 29]


In [9]:
def  find_subset_sum_trim(S, t, factor):
    L_prev = [0]  #相当于L(i-1)
    for i in range(1, len(S)):
        L = []
        for l in L_prev:  #L(i-1)+x(i)
            L.append(l + S[i])
        L_cur = []  #相当于L(i)
        p = 0
        q = 0
        while p < len(L_prev) or q < len(L): #将L(i-1) 与 L(i-1) + x(i)两个数组的元素合并，同时保证合并后元素不重复且排好序
            if p < len(L_prev) and q < len(L):
                k = min(L_prev[p], L[q])
                if  k not in L_cur:   #由于会有剪裁,因此不考虑元素大于t的情况
                    L_cur.append(k)
                if k == L_prev[p]:
                    p += 1
                else:
                    q += 1
            else:  #将剩余数组中的元素加入L_cur
                LL = []
                if p >= len(L_prev):
                    LL = L
                else:
                    LL = L_prev 
                for l in LL:
                    if l not in L_cur : #由于会有剪裁,因此不考虑元素大于t的情况
                        L_cur.append(l)
                break
        L_prev = L_cur 
        L_prev = trim(L_prev, factor / 2*len(L_prev))
        L_cur = [l for l in L_prev if l <= t] #去除大于t的元素
        L_prev = L_cur
        print(L_prev)
    return L_prev[len(L_prev) - 1]
S = [104, 102, 201, 101]
t = 308
close_to_t = find_subset_sum_trim(S, t, 0.4)
print("subset sum with trim is ", close_to_t)

[0, 102]
[0, 102, 201]
[0, 101, 302]
subset sum with trim is  302
